In [61]:
import os
import shutil
import sh
import subprocess
from pyspark.sql.functions import *

In [62]:
directory = '/user/marekdragan/New_Tweets/'
file = '*.json'
path = directory + file

In [63]:
tweets_df = spark.read.json(path)

In [64]:
tweets_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [65]:
tweets_df_l1 = tweets_df.select("id", "lang", "created_at", "user.screen_name", \
                                #"user.name",\
                                "user.location", "retweet_count", "text")
#You MAY run out of memory with caching - use with caution
#tweets_df_l1.cache()
type(tweets_df_l1)

pyspark.sql.dataframe.DataFrame

In [66]:
tweets_df_l1.show(10)

+------------------+----+--------------------+---------------+------------------+-------------+--------------------+
|                id|lang|          created_at|    screen_name|          location|retweet_count|                text|
+------------------+----+--------------------+---------------+------------------+-------------+--------------------+
|851505580489547776|  en|Mon Apr 10 18:42:...|   USPressWorld|        Etats-Unis|            0|Illinois Judge Sh...|
|851505581613559808|  en|Mon Apr 10 18:42:...|         TRIYZ_|              null|            0|RT @Smooky_Montan...|
|851505581806338048|  en|Mon Apr 10 18:42:...| KristyBerridge| Cairns, Australia|            0|RT @LIONPubs: Sav...|
|851505584213983235|  en|Mon Apr 10 18:42:...|   kylah_hudson|              hell|            0|RT @SLAYLONIE: yo...|
|851505583995924480|  en|Mon Apr 10 18:42:...|       Suntimes|       Chicago, IL|            0|Dancer-actress Ch...|
|851505583765245955|  en|Mon Apr 10 18:42:...|    5tweetstogo|  

In [67]:
tweets_df_l2 = tweets_df_l1.where("lower(text) like '%Hyde Park%' \
                        or lower(text) like '%uchicago%' \
                        or lower(text) like '%regenstein%' \
                        or lower(text) like '%the midway%' \
                        or (text) like '%Harper Library%' \
                        or (text) like '%Humanities Day%' \
                        or lower(text) like '%gleacher%'")
tweets_df_l2.count()

3662

In [76]:
tweets_df_l2 = tweets_df_l1.where("lower(text) like '%uchicago%' \
                        or lower(text) like '%hyde park%'\
                        or (text) like '%Booth%'\
                        or lower(text) like '%regenstein%' \
                        or lower(text) like '%the midway%' \
                        or (text) like '%u of c%' \
                        or lower(text) like '%gleacher%'")
tweets_df_l2.count()

7234

In [47]:
tweets_df_l2.select("text").show(30, truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|RT @MEF_Psych_Dept: This Thursday, Rob Booth will give a talk to the Cognitive Neuroscience group at Istanbul University Faculty of Med…    |
|#Bulls #Card: 1996-1997 Michael Jordan Bowman's Best Chicago Bulls #80 YuChicago#3352 https://t.co/avpubEOiMN #Cards https://t.co/xpRhjfO2J2|
|A man, 18, was shot and wounded in Hyde Park: https://t.co/C2oztvcvQJ                                                                       |
|@Steven_Hyden Just realized that, had they been craven/with Ticketmaster, Pearl Jam could have closed Boston Garden + Chicago Stadium.      |

In [48]:
tweets_df_l3 = tweets_df_l2.withColumn("text_clean", regexp_replace("text", "[\\r\\n\\t]", " "))
#withColumn("text_clean", trim(lower(col("text_clean"))))

In [49]:
tweets_df_l3.select("text_clean").show(5, truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text_clean                                                                                                                                  |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|RT @MEF_Psych_Dept: This Thursday, Rob Booth will give a talk to the Cognitive Neuroscience group at Istanbul University Faculty of Med…    |
|#Bulls #Card: 1996-1997 Michael Jordan Bowman's Best Chicago Bulls #80 YuChicago#3352 https://t.co/avpubEOiMN #Cards https://t.co/xpRhjfO2J2|
|A man, 18, was shot and wounded in Hyde Park: https://t.co/C2oztvcvQJ                                                                       |
|@Steven_Hyden Just realized that, had they been craven/with Ticketmaster, Pearl Jam could have closed Boston Garden + Chicago Stadium.      |

In [50]:
tweets_df_l3.count()

9864

In [51]:
tweets_df_l4 = tweets_df_l3.drop('text')

In [52]:
tweets_df_l4.show(5)

+------------------+----+--------------------+---------------+-----------------+-------------+--------------------+
|                id|lang|          created_at|    screen_name|         location|retweet_count|          text_clean|
+------------------+----+--------------------+---------------+-----------------+-------------+--------------------+
|851508016549679104|  en|Mon Apr 10 18:51:...|        ccerenk|             null|            0|RT @MEF_Psych_Dep...|
|851512674668036096|  en|Mon Apr 10 19:10:...|  Jonelle_Donis|             null|            0|#Bulls #Card: 199...|
|851513886641618944|  en|Mon Apr 10 19:15:...|BauerJournalism|Chicago, Illinois|            0|A man, 18, was sh...|
|851514663384776704|  en|Mon Apr 10 19:18:...|      KDonhoops|    Lafayette, IN|            0|@Steven_Hyden Jus...|
|851525034166321152|  en|Mon Apr 10 19:59:...|sunneversets100|             null|            0|RT @JimAngone: @r...|
+------------------+----+--------------------+---------------+----------

In [55]:
# Save data
twit_out_hdfs = "hdfs:///user/jfnavarr/twitter_out"
twit_out_linx_dir = "/home/jfnavarr/temp/"
twit_out_linx_file = "twitter_out.txt"
twit_out_linx = twit_out_linx_dir + twit_out_linx_file

In [56]:
try:
    sh.hdfs('dfs','-rmr',twit_out_hdfs)
except:
    print(twit_out_hdfs+' *** Does not exist ***')

In [ ]:
#tweets_df_l4.write.format("com.databricks.spark.csv").option("header", "false").option("delimiter", "\t").save(twit_out_hdfs)
tweets_df_l4.write.format("com.databricks.spark.csv").option("header", "true").option("delimiter", "\t").save(twit_out_hdfs)

In [ ]:
try:
    print(sh.hdfs('dfs','-ls',twit_out_hdfs))
except:
    print(twit_out_hdfs+' *** Does not exist ***')

In [ ]:
def run_cmd(args_list):
    print('Running system command: {0}'.format(' '.join(args_list)))
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE,
            stderr=subprocess.PIPE)
    (output, errors) = proc.communicate()
    if proc.returncode:
        raise RuntimeError(
            'Error running command: %s. Return code: %d, Error: %s' % (join(args_list), proc.returncode, errors))
    return (output, errors)

In [ ]:
(out, errors)= run_cmd(['hadoop', 'fs', '-getmerge', twit_out_hdfs, twit_out_linx])

In [ ]:
os.listdir(twit_out_linx_dir)